In [1]:
from src.models import hmm, decoders
import numpy as np

In [2]:
obs_states = ['regulatory', 'regulatory-potential']
hid_states = ['encode_atac', 'atac']

# Import the HMM input data for progenitor cardiomyocytes (prefix: prog_cm)
prog_cm_data = np.load('./data/ProjectDeliverable-ProgenitorCMs.npz')
prob_prior = prog_cm_data['prior_probabilities']
prob_trans = prog_cm_data['transition_probabilities']
prob_emiss = prog_cm_data['emission_probabilities']

HMM = hmm.HiddenMarkovModel(observation_states = obs_states,
                           hidden_states = hid_states,
                           prior_probabilities = prob_prior,
                           transition_probabilities = prob_trans,
                           emission_probabilities = prob_emiss)

In [3]:
# Instantiate submodule class models.ViterbiAlgorithm with the progenitor cardiomyocyte's HMM
prog_cm_viterbi_instance = decoders.ViterbiAlgorithm(HMM)
# prog_cm_viterbi_instance.best_hidden_state_sequence(prog_cm_data['observation_states'])

In [4]:
# Decode the hidden states (i.e., CRE selection strategy) for the progenitor CMs and evaluate the model performace
evaluate_viterbi_decoder_using_observation_states_of_prog_cm = prog_cm_viterbi_instance.best_hidden_state_sequence(prog_cm_data['observation_states'])

In [5]:
evaluate_viterbi_decoder_using_observation_states_of_prog_cm

(array(['atac', 'atac', 'encode_atac', 'encode_atac', 'encode_atac',
        'atac', 'encode_atac', 'atac', 'atac', 'atac'], dtype='<U11'),
 3.019898880000002e-06)

In [6]:
decode_observation_states = prog_cm_data['observation_states']
decode_observation_states

array(['regulatory', 'regulatory-potential', 'regulatory-potential',
       'regulatory-potential', 'regulatory', 'regulatory-potential',
       'regulatory', 'regulatory', 'regulatory', 'regulatory-potential'],
      dtype='<U20')

In [7]:
print("Prior probabilities")
print(prob_prior)
print("Transmission probabilities")
print(prob_trans)
print("Emission probabilities")
print(prob_emiss)

Prior probabilities
[0.5 0.5]
Transmission probabilities
[[0.4 0.6]
 [0.5 0.5]]
Emission probabilities
[[0.6 0.4]
 [0.8 0.2]]


In [8]:
def trellis_node_step (prev_delta, prob_emiss, prob_trans, observed_state):
    possible_state = np.multiply(prev_delta, prob_trans.T)
    probable_state = np.argmax(possible_state, axis = 1)
    
    probable_state_vals = np.amax(possible_state, axis = 1)
    
    _delta = np.multiply(probable_state_vals, prob_emiss)
    new_delta = _delta[:, observed_state]
    
    return new_delta, probable_state

In [9]:
# This is the probability of each hidden state
hidden_state_prob = np.zeros((len(decode_observation_states), 
                              len(HMM.hidden_states)))

# Given the starting probabilites, what are the possible emission probabilies?
# calculate the intial delta
delta = np.multiply(prob_prior, prob_emiss)
hidden_state_prob[0,:] = delta[:, HMM.observation_states_dict[decode_observation_states[0]]]



all_prev_states = np.zeros((len(decode_observation_states), 
                              len(HMM.hidden_states)))
all_prev_states[0,:] = [hidden_state_index for hidden_state_index in range(len(HMM.hidden_states))]

# Loop through each observation state node:
for trellis_node in range(1, len(decode_observation_states)): 
    # Get the hidden state of the previous step
    prev_delta = hidden_state_prob[trellis_node-1, :]
    observed_state = HMM.observation_states_dict[decode_observation_states[trellis_node]]
    
    new_delta, prev_states = trellis_node_step(prev_delta, prob_emiss, prob_trans, observed_state)
    hidden_state_prob[trellis_node, :] = new_delta
    all_prev_states[trellis_node, :] = prev_states

In [10]:
# Get highest probability sequence
state = np.argmax(new_delta)
sequence_prob = np.amax(new_delta)
best_path = []

idx = 0
for prev_states in all_prev_states[::-1]:
    print(prev_states)
    print(idx)
    idx += 1
    state = prev_states[int(state)]
    best_path.append(state)
hidden_state_path = np.array([HMM.hidden_states_dict[i] for i in best_path[::-1]])

[1. 1.]
0
[1. 1.]
1
[1. 1.]
2
[0. 0.]
3
[1. 1.]
4
[0. 0.]
5
[0. 0.]
6
[0. 0.]
7
[1. 1.]
8
[0. 1.]
9


In [10]:
hidden_state_path

NameError: name 'hidden_state_path' is not defined

In [17]:
best_path

[1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]

In [12]:
all_prev_states

array([[0., 1.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [1., 1.],
       [1., 1.]])